#  Introduction

Proximity analysis in GIS such as measuring distance between points and creating buffers are very useful analysis. In this notebook, i tried to do the aforementioned analysis using data from the US Environmental Protection agency that tracks chemicals in Pennsylvania, USA. The data was obtained from kaggle.com

In [27]:
import folium
import webbrowser
from folium import Marker, GeoJson
from folium.plugins import HeatMap

import pandas as pd
import geopandas as gpd

### First, load the dataset and and read them using pandas and geopandas

In [8]:
release = gpd.read_file("C:/Users/Petra/Downloads/toxic_release_pennsylvania/toxic_release_pennsylvania.shp")
release.head()

,YEAR,CITY,COUNTY,ST,LATITUDE,LONGITUDE,CHEMICAL,UNIT_OF_ME,TOTAL_RELE,geometry
0,2016,PHILADELPHIA,PHILADELPHIA,PA,40.005901,-75.072103,FORMIC ACID,Pounds,0.160,POINT (2718560.227 256380.179)
1,2016,PHILADELPHIA,PHILADELPHIA,PA,39.920120,-75.146410,ETHYLENE GLYCOL,Pounds,13353.480,POINT (2698674.606 224522.905)
2,2016,PHILADELPHIA,PHILADELPHIA,PA,40.023880,-75.220450,CERTAIN GLYCOL ETHERS,Pounds,104.135,POINT (2676833.394 261701.856)
3,2016,PHILADELPHIA,PHILADELPHIA,PA,39.913540,-75.198890,LEAD COMPOUNDS,Pounds,1730.280,POINT (2684030.004 221697.388)
4,2016,PHILADELPHIA,PHILADELPHIA,PA,39.913540,-75.198890,BENZENE,Pounds,39863.290,POINT (2684030.004 221697.388)


Next, since I would have to work with dataset involving readings from air quality monitoring stations in the same city, it is important to also load and read such data

In [9]:
stations = gpd.read_file("C:/Users/Petra/Downloads/PhillyHealth_Air_Monitoring_Stations/PhillyHealth_Air_Monitoring_Stations.shp")
stations.head()

,SITE_NAME,ADDRESS,BLACK_CARB,ULTRAFINE_,CO,SO2,OZONE,NO2,NOY_NO,PM10,...,PAMS_VOC,TSP_11101,TSP_METALS,TSP_LEAD,TOXICS_TO1,MET,COMMUNITY_,LATITUDE,LONGITUDE,geometry
0,LAB,1501 East Lycoming Avenue,N,N,Y,N,Y,Y,Y,N,...,Y,N,Y,N,y,N,N,40.008606,-75.097624,POINT (2711384.641 257149.310)
1,ROX,Eva and Dearnley Streets,N,N,N,N,N,N,N,N,...,N,N,Y,N,Y,N,N,40.050461,-75.236966,POINT (2671934.290 271248.900)
2,NEA,Grant Avenue and Ashton Street,N,N,N,N,Y,N,N,N,...,N,N,N,N,N,Y,N,40.072073,-75.013128,POINT (2734326.638 280980.247)
3,CHS,500 South Broad Street,N,N,N,N,N,N,N,N,...,N,N,Y,N,Y,N,N,39.944510,-75.165442,POINT (2693078.580 233247.101)
4,NEW,2861 Lewis Street,N,N,Y,Y,Y,N,Y,Y,...,N,Y,N,Y,N,Y,N,39.991688,-75.080378,POINT (2716399.773 251134.976)


# Measuring distance

### To measure distances between points from two different GeoDataFrames, it is important to make sure they use the same coordinate reference system (CRS). 

In [11]:
print(stations.crs)
print(release.crs)

epsg:2272
epsg:2272


### Now that it is confirmed that both shapefiles uses the same CRS, one can now go ahead to conduct distance calculations in the GeoDataFrame using recent_release and Stations

In [15]:
# Select one release incident
recent_release = release.iloc[360]

# measure the distance from release to each station

distance = stations.geometry.distance(recent_release.geometry)
print(distance)

0     44778.509761
1     51006.456589
2     77744.509207
3     14672.170878
4     43753.554393
5      4711.658655
6     23197.430858
7     12072.823097
8     79081.825506
9      3780.623591
10    27577.474903
11    19818.381002
dtype: float64


### These distances are in Feet since the epsg 2272 is in NAD83. I would prefer to work in meters. Thus a conversion has to be carried out. 1ft = 0.3048

In [17]:
recent_release = release.iloc[360]
distance_meters = stations.geometry.distance(recent_release.geometry)*0.3048
print(distance_meters)

0     13648.489775
1     15546.767968
2     23696.526406
3      4472.077684
4     13336.083379
5      1436.113558
6      7070.576926
7      3679.796480
8     24104.140414
9      1152.334070
10     8405.614351
11     6040.642529
dtype: float64


### Now, having estimated the distance of each release to a station, one can now go ahead to calculate the mean distance to each monitoring station

In [18]:
print('Mean distance to monitoring station: {} meters'.format(distance.mean()))

Mean distance to monitoring station: 10215.763628399733 meters


### I can also determine the closest monitoring station

In [23]:
print('Closest monitoring station({} meters):'.format(distance_meters.min()))
print(stations.iloc[distance.idxmin()][["ADDRESS", "LATITUDE", "LONGITUDE"]])

Closest monitoring station(1152.334070401604 meters):
ADDRESS      3100 Penrose Ferry Road
LATITUDE                    39.91279
LONGITUDE                 -75.185448
Name: 9, dtype: object


# Now I can also do buffer analysis

### In this instance I would like to create a 2 Km buffer around each of the 12 air monitoring stations (2*1609.344 meters)

In [24]:
two_km_buffer = stations.geometry.buffer(2*1609,344)
two_km_buffer.head()

0    POLYGON ((2714602.641 257149.310, 2714602.607 ...
1    POLYGON ((2675152.290 271248.900, 2675152.256 ...
2    POLYGON ((2737544.638 280980.247, 2737544.605 ...
3    POLYGON ((2696296.580 233247.101, 2696296.546 ...
4    POLYGON ((2719617.773 251134.976, 2719617.739 ...
dtype: geometry

### Next I can use the folium.GeoJson() to plot each polygon to a map. Before plotting I would have to convert the CRS to epsg 4326 since that is required in folium 

In [31]:
m = folium.Map(location=[39.9526, -75.1652], zoom_start=11)
HeatMap(data=release[['LATITUDE', 'LONGITUDE']], radius=15).add_to(m)
for idx, row in stations.iterrows():
    Marker([row['LATITUDE'], row['LONGITUDE']]).add_to(m)
    
# Now I can plot each polygon on the map
GeoJson(two_km_buffer.to_crs(epsg=4326)).add_to(m)
print(m)

In [36]:
import folium
import webbrowser

map = folium.Map(location=[39.9526, -75.1652], zoom_start=11)
HeatMap(data=release[['LATITUDE', 'LONGITUDE']], radius=15).add_to(map)
for idx, row in stations.iterrows():
    Marker([row['LATITUDE'], row['LONGITUDE']]).add_to(map)
class Map:
    def __init__(self, center, zoom_start):
        self.center = center
        self.zoom_start = zoom_start
    
    def showMap(self):
        #Create the map
        my_map = folium.Map(location = self.center, zoom_start = self.zoom_start)

        #Display the map
        my_map.save("map.html")
        webbrowser.open("map.html")

#Define coordinates of where we want to center our map
coords = [39.9526, -75.1652]
map = Map(center = coords, zoom_start = 11)
map.showMap()

In [37]:
my_union = two_km_buffer.geometry.unary_union
print('Type:', type(my_union))

Type: <class 'shapely.geometry.multipolygon.MultiPolygon'>
